In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Play a Chess Game

In [2]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("PyPlot")
using PyPlot
# using display
# Pkg.add("NBInclude")
using NBInclude

In [4]:
@nbinclude("Random Chess.ipynb")

playMove (generic function with 1 method)

In [5]:
@nbinclude("Chess Games Save.ipynb")

saveGameToPGN (generic function with 1 method)

In [6]:
@nbinclude("ValidateUserInput.ipynb")

is_legal_move (generic function with 1 method)

`printGame` function prints the board to standard output. Settings can be set here. 

In [7]:
function printGame(game)
    display(board(game))
#     pprint(board(game), color = true)
end

printGame (generic function with 1 method)

`printWin(game)` prints the result of the game as a text message.

In [8]:
function printWin(board)
    if ischeckmate(board)
        color = sidetomove(board)
        color == "WHITE" ? color = "BLACK" : color = "WHITE"
        print(color * " wins by checkmate" )
    elseif isstalemate(board)
        print("Stalemate")
    elseif ismaterialdraw(board)
        print("Draw by insufficient material")
    elseif isrule50draw(board)
        print("Draw by 50 move draw")
    end
end

printWin (generic function with 1 method)

In [9]:
function playGame(color, seed)
    Random.seed!(seed)
    # validation
    game = SimpleGame()
    printGame(game)
    setGameHeaders(game)
    println("what's your name?")
    username = readline()
    # Offset move if player is black
    if color != "black" && color != "white"
        print("Invalid color. Choose white or black")   
        return
    end
    
    if color == "black"
        setheadervalue!(game, "Black", username)
        playRandomMove(game)
        printGame(game)
    elseif color == "white"
        setheadervalue!(game, "White", username)
    end

    while true
        # User Move
        if !isterminal(game)
            println("Make a move")
            userInput = readline()
            userMove = movefromstring(userInput)
            if(userMove == nothing)
                println("Invalid User Input. Please use the format [source] [destination]. Example(e2e4)")
                continue
            end
            if(is_legal_move(game, userMove))
                domove!(game, userMove)
            else
                println("Invalid Chess move. Please make a other move.")
                continue
            end
            # IJulia.clear_output(true)
            # sleep(1)
            printGame(game)
        else
            break
        end
        
        # Delay
        sleep(1)
        
        # Engine Move
        if !isterminal(game)
            playRandomMove(game)
            # IJulia.clear_output(true)
            # sleep(1)
            printGame(game)
        else
            break
        end
    end
    saveGameToPGN(game)
    printWin(board(game))
    return game
end

playGame (generic function with 1 method)

In [ ]:
playGame("white", 1233)

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

what's your name?
stdin> Luca
Make a move
stdin> e2e4


Board (rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (rnbqkbnr/pppppp1p/8/6p1/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  -  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  p  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Make a move
